In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
pwd

'/content'

In [ ]:
!dir

sample_data


In [ ]:
!mv "/Clean_Connect_Mobile__Attrition_Data_file.csv" "/content"

In [ ]:
df_clean=pd.read_csv('Clean_Connect_Mobile__Attrition_Data_file.csv')

In [ ]:
df_clean.head()

,active_cust,income,expenditure,Num_complaints,number_plan_changes,relocated,monthly_bill,technical_issues_per_month
0,0,1586,1268.8,4,1,0,121,4
1,0,1581,1264.8,4,1,0,133,4
2,0,1594,1275.2,4,1,0,118,4
3,0,1594,1275.2,4,1,0,123,4
4,1,1609,1287.2,4,1,0,177,4


In [ ]:
X=df_clean.drop('active_cust',axis=1) #features
y=df_clean['active_cust'] #label


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42) #60% train,40%testing
# X_val,X_test,y_val,y_test = train_test_split(X_test,y_test,test_size=0.5,random_state=42)

In [ ]:
X_train.to_csv('train_features.csv', index=False)
# X_val.to_csv('val_features.csv', index=False)
X_test.to_csv('test_features.csv', index=False)

y_train.to_csv('train_labels.csv', index=False)
# y_val.to_csv('val_labels.csv', index=False)
y_test.to_csv('test_labels.csv', index=False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier


In [ ]:
def print_result(results):
    print("Best Params={}\n".format(results.best_params_))
    
    means=results.cv_results_['mean_test_score']
    stds=results.cv_results_['std_test_score']
    
    for mean,std,params in zip(means,stds,results.cv_results_['params']):
        print("{} (+/-{}) for {} ".format(round(mean, 3), round(std * 2, 3), params))

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
tr_features=pd.read_csv('train_features.csv')
tr_labels=pd.read_csv('train_labels.csv')

In [ ]:
rf=RandomForestClassifier()
parameters = {
    'n_estimators': [5, 50, 250],
    'max_depth': [2, 4, 8, 16, 32]
}
cv=GridSearchCV(rf,parameters,cv=5)
cv.fit(tr_features,tr_labels.values.ravel())

print_result(cv)


Best Params={'max_depth': 16, 'n_estimators': 250}

0.698 (+/-0.048) for {'max_depth': 2, 'n_estimators': 5} 
0.692 (+/-0.009) for {'max_depth': 2, 'n_estimators': 50} 
0.69 (+/-0.008) for {'max_depth': 2, 'n_estimators': 250} 
0.778 (+/-0.035) for {'max_depth': 4, 'n_estimators': 5} 
0.794 (+/-0.012) for {'max_depth': 4, 'n_estimators': 50} 
0.798 (+/-0.02) for {'max_depth': 4, 'n_estimators': 250} 
0.829 (+/-0.007) for {'max_depth': 8, 'n_estimators': 5} 
0.839 (+/-0.005) for {'max_depth': 8, 'n_estimators': 50} 
0.839 (+/-0.005) for {'max_depth': 8, 'n_estimators': 250} 
0.836 (+/-0.005) for {'max_depth': 16, 'n_estimators': 5} 
0.844 (+/-0.005) for {'max_depth': 16, 'n_estimators': 50} 
0.846 (+/-0.004) for {'max_depth': 16, 'n_estimators': 250} 
0.808 (+/-0.004) for {'max_depth': 32, 'n_estimators': 5} 
0.819 (+/-0.003) for {'max_depth': 32, 'n_estimators': 50} 
0.821 (+/-0.005) for {'max_depth': 32, 'n_estimators': 250} 


In [ ]:
cv.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=16, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=250,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
import joblib

In [ ]:
joblib.dump(cv.best_estimator_,'RF_model.pkl')

['RF_model.pkl']

In [ ]:
model=joblib.load('RF_model.pkl')

In [ ]:
model

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=16, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=250,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from time import time

In [ ]:
test_features=pd.read_csv('test_features.csv')
test_label = pd.read_csv('test_labels.csv')

In [ ]:
def evaluate_model(name, model, features, labels):
    start = time()
    pred = model.predict(features)
    print(pred)
    print(len(pred))
    end = time()
    accuracy = round(accuracy_score(labels, pred), 3)
    precision = round(precision_score(labels, pred), 3)
    recall = round(recall_score(labels, pred), 3)
    print('{} -- Accuracy: {} / Precision: {} / Recall: {} / Latency: {}ms'.format(name,
                                                                                   accuracy,
                                                                                   precision,
                                                                                   recall,
                                                                                   round((end - start)*1000, 1)))

In [ ]:
evaluate_model("RF",model,test_features,test_label)

[1 1 1 ... 1 1 0]
40000
RF -- Accuracy: 0.847 / Precision: 0.855 / Recall: 0.888 / Latency: 1750.2ms


In [ ]:
test_label

,active_cust
0,1
1,1
2,1
3,1
4,0
...,...
39995,0
39996,0
39997,1
39998,1


In [ ]:
df_clean.head()

,active_cust,income,expenditure,Num_complaints,number_plan_changes,relocated,monthly_bill,technical_issues_per_month
0,0,1586,1268.8,4,1,0,121,4
1,0,1581,1264.8,4,1,0,133,4
2,0,1594,1275.2,4,1,0,118,4
3,0,1594,1275.2,4,1,0,123,4
4,1,1609,1287.2,4,1,0,177,4


In [ ]:
rf2=RandomForestClassifier(max_depth=16,n_estimators=250)

In [ ]:
tr_labels

,active_cust
0,1
1,1
2,0
3,1
4,1
...,...
59995,1
59996,0
59997,1
59998,1


In [ ]:
rf2.fit(tr_features,tr_labels)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=16, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=250,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
pred=rf2.predict(test_features)

print(accuracy_score(test_label,pred))

0.84705


In [ ]:
print(pred)

[1 1 1 ... 1 1 0]


In [ ]:
print(test_label)

       active_cust
0                1
1                1
2                1
3                1
4                0
...            ...
39995            0
39996            0
39997            1
39998            1
39999            0

[40000 rows x 1 columns]


In [ ]:
pred.shape

(40000,)

In [ ]:
test_features.shape

(40000, 7)

In [ ]:
test_features

,income,expenditure,Num_complaints,number_plan_changes,relocated,monthly_bill,technical_issues_per_month
0,1611,1288.80,4,2,0,45,1
1,1728,1382.40,4,0,0,36,1
2,1680,1092.00,4,0,0,168,1
3,1617,1293.60,4,0,0,90,4
4,1737,1389.60,3,2,0,84,4
...,...,...,...,...,...,...,...
39995,1741,1392.80,4,3,0,83,4
39996,1585,1030.25,4,3,1,131,4
39997,1633,1306.40,4,1,0,83,4
39998,1642,1313.60,4,1,0,132,4


In [ ]:
test_features['active_cust']=test_label

In [ ]:
test_features.head()

,income,expenditure,Num_complaints,number_plan_changes,relocated,monthly_bill,technical_issues_per_month,active_cust
0,1611,1288.8,4,2,0,45,1,1
1,1728,1382.4,4,0,0,36,1,1
2,1680,1092.0,4,0,0,168,1,1
3,1617,1293.6,4,0,0,90,4,1
4,1737,1389.6,3,2,0,84,4,0


In [ ]:
test_features['churn']=pred

In [ ]:
test_features.head()

,income,expenditure,Num_complaints,number_plan_changes,relocated,monthly_bill,technical_issues_per_month,active_cust,churn
0,1611,1288.8,4,2,0,45,1,1,1
1,1728,1382.4,4,0,0,36,1,1,1
2,1680,1092.0,4,0,0,168,1,1,1
3,1617,1293.6,4,0,0,90,4,1,1
4,1737,1389.6,3,2,0,84,4,0,1
